# Chatbot Modeling

## Import Libs

In [1]:
# General
import json
import string
import pickle
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import LabelEncoder

# Deep Learning
import tensorflow as tf

# Word Preprocessing
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt') # Only needed for first time usage of NLTK

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load Files for Preprocessing and Modeling

### Intent

In [2]:
# Load json file
f = open('intent/intent.json', 'r')
intent_json = json.load(f)

In [3]:
# Create list from json
input = []
intent = []

for i in range(len(intent_json['intents'])):
    for user_input in intent_json['intents'][i]['input']:
        input.append(user_input)
        intent.append(intent_json['intents'][i]['intent'])

In [4]:
# Create dataframe from json
df = pd.DataFrame({
    'input': input,
    'intent' : intent
    # 'response' : response
})
df.head()

,input,intent
0,halo,sapa
1,hei,sapa
2,hey,sapa
3,hello,sapa
4,hallo,sapa


### Stopwords

In [5]:
# Define stemmer and stopword
factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

In [6]:
stopwords.remove('ok')
stopwords.remove('oh')
stopwords.remove('tidak')
stopwords.remove('ya')

### Slang

In [7]:
# slang = pd.read_csv('lexicon/slang ke semi baku.csv')

# slang_replace = {}
# for i, row in enumerate(slang['slang']):
#     slang_replace[row] = slang['formal'].iloc[i]

In [8]:
# baku = pd.read_csv('lexicon/slang ke baku.csv')

# std_word_replace = {}
# for i, row in enumerate(baku['slang']):
#     std_word_replace[row] = baku['baku'].iloc[i]

In [9]:
baku = pd.read_csv('lexicon/baku.csv')

In [10]:
std_word_replace = {}
for i, row in enumerate(baku['slang']):
    std_word_replace[row] = baku['baku'].iloc[i]

In [11]:
std_word_replace['gak']

'tidak'

## Preprocessing

In [12]:
# Create text cleaning function
def clean_text(text):
    new_text = []
    text = text.lower() # Lowercase
    # Loop each word in a sentence
    for kata in text.split(): 
        # Keep word not in slang or standard word
        if kata not in std_word_replace: 
            new_text.append(kata) 
        # Replace non-formal word with standard word
        elif kata in std_word_replace:
            new_text+=std_word_replace[kata].split() 
    # Join words without stopwords after stemming
    new_text = ' '.join(
        stemmer.stem(word) for word in new_text if word not in stopwords
    )
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    return new_text

In [13]:
kalimat = 'Aku laper banget gatau maunya makan jeruk apa lagi meniru-niru... daah ngelamar'
clean_text(kalimat)

'aku lapar banget tidak tahu mau makan jeruk apa tiru deh lamar'

In [14]:
df['clean_input'] = df['input'].apply(clean_text)
df['clean_input'].head()

0     halo
1      hei
2      hai
3    hello
4     halo
Name: clean_input, dtype: object

## Create Corpus

In [15]:
# Create corpus
words = set([
    word for word in df['clean_input'] for word in word_tokenize(word)
    ]
)

In [16]:
corpus_size = len(words)

In [17]:
# Calculate word length of each row
df['length'] = df['clean_input'].apply(word_tokenize).apply(len)

In [18]:
sequence_length = int(round(df['length'].max(),0))

In [19]:
corpus_size, sequence_length

(352, 8)

## Label Encoding

In [20]:
# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(df['intent'])
y_train = tf.keras.utils.to_categorical(y_train)

In [21]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [22]:
len(le.classes_)

22

## Modeling

### Text Vectorization

In [23]:
textvect = tf.keras.layers.TextVectorization(
    max_tokens=corpus_size,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=sequence_length
)
textvect.adapt(df['clean_input'])

In [24]:
tes = 'saya mau daftar rekrutmen'
textvect(clean_text(tes))

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([22, 12, 11,  0,  0,  0,  0,  0], dtype=int64)>

### Word Embedding

In [25]:
embedding = tf.keras.layers.Embedding(
    input_dim=corpus_size,
    output_dim=16,
    input_length=sequence_length,
    embeddings_initializer='uniform'
)

In [26]:
embedding(textvect(clean_text(tes)))

<tf.Tensor: shape=(8, 16), dtype=float32, numpy=
array([[-0.00431864, -0.00600551,  0.04074701, -0.03494113, -0.02862122,
         0.03925189, -0.01519897,  0.03140778,  0.04204262,  0.03446306,
        -0.01218202,  0.0471083 , -0.04838435,  0.02143338,  0.0086353 ,
         0.04718659],
       [-0.01082016,  0.03313905,  0.01812093, -0.02978057, -0.0409055 ,
         0.04579781,  0.00854098,  0.00857363,  0.04625325,  0.04924398,
        -0.04154586,  0.03287314,  0.00188011,  0.03037728, -0.00063781,
        -0.02044158],
       [ 0.00138515,  0.04211135,  0.04754808, -0.01087421, -0.02151618,
         0.04651405, -0.00453151, -0.048998  , -0.04395633,  0.02800468,
        -0.02774354,  0.02561859, -0.00235011, -0.00193802, -0.01057472,
        -0.02505268],
       [ 0.00180967, -0.00501917, -0.03107287,  0.02208437,  0.02426105,
         0.01398769,  0.02986543, -0.02855657,  0.00790197, -0.00058115,
        -0.01238186, -0.0148887 ,  0.0128679 ,  0.0274126 ,  0.02289916,
         

### Layering

In [27]:
# Create model
input = tf.keras.layers.Input(shape=(1,), dtype='string')
hidden_layer = textvect(input)
hidden_layer = embedding(hidden_layer)
hidden_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16))(hidden_layer)
output = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(hidden_layer)
model = tf.keras.Model(inputs=input, outputs=output)

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

### Training

In [28]:
hist = model.fit(df['clean_input'], y_train, epochs=100, verbose=0)
pd.DataFrame(hist.history).tail()

,loss,categorical_accuracy
95,0.032032,0.998168
96,0.029016,0.998168
97,0.025745,1.000000
98,0.025300,1.000000
99,0.024718,1.000000


### Evaluating Model

In [29]:
model.evaluate(df['clean_input'], y_train)

18/18 [==============================] - 2s 3ms/step - loss: 0.0235 - categorical_accuracy: 1.0000


[0.023468678817152977, 1.0]

In [30]:
y_train_df = pd.DataFrame(y_train, columns=le.classes_)
y_train_df['intent'] = y_train_df.idxmax(axis=1)

model_pred = model.predict(df['clean_input'])
model_pred = pd.DataFrame(model_pred, columns=le.classes_)
model_pred['intent'] = model_pred.idxmax(axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_train_df['intent'], model_pred['intent']))

                  precision    recall  f1-score   support

              CV       1.00      1.00      1.00        12
           biaya       1.00      1.00      1.00        28
          daftar       1.00      1.00      1.00        22
         dokumen       1.00      1.00      1.00        23
    error upload       1.00      1.00      1.00        13
             ktp       1.00      1.00      1.00        17
     link sosmed       1.00      1.00      1.00        38
        lowongan       1.00      1.00      1.00        17
   lupa password       1.00      1.00      1.00        24
            nama       1.00      1.00      1.00         9
        pas foto       1.00      1.00      1.00        23
       pengantar       1.00      1.00      1.00        16
         penutup       1.00      1.00      1.00        48
   qualification       1.00      1.00      1.00        21
responsibilities       1.00      1.00      1.00        33
          salary       1.00      1.00      1.00        38
            s

### Function Covering End-to-End Process

In [31]:
def bot_response(text):
    """Take text as function input then predict using model. Return response based on highest probability using numpy argmax    
    """
    text = clean_text(text)
    pred = model.predict([text])
    res = le.classes_[pred.argmax()] # Get the index with highest probability
    i = 0
    try:
        if textvect(text).numpy().max() > 1: # If the input is known word(s)
            while i < len(intent_json['intents']):
                if res == intent_json['intents'][i]['intent']:
                    response = intent_json['intents'][i]['response']
                    break
                else:
                    i+=1
        else: # If only unknown word(s)
            response = ['Maaf, Kak. Aku tidak mengerti...']
    except: # If empty string or any error occured
        response = ['Maaf, Kak. Aku tidak mengerti...']

    # For debugging only
    dict_temp = []
    for i in range(len(pred[0])):
        temp = {le.classes_[i]: pred[0][i]}
        dict_temp.append(temp)
    print(dict_temp)
    print(le.classes_[pred.argmax()])

    return print(np.random.choice(response))

In [32]:
tes1 = 'aku mau daftar rekrutmen'
clean_text(tes1)

'aku mau daftar rekrutmen'

In [33]:
model.predict([clean_text(tes1)])

array([[3.6499318e-06, 2.0937042e-04, 9.5738101e-01, 2.6261874e-03,
        1.7256076e-06, 1.4831529e-04, 5.6147121e-05, 1.7104638e-04,
        2.5318521e-05, 9.2375744e-04, 1.9316751e-04, 3.4116172e-06,
        3.6567333e-06, 3.7070073e-02, 7.1733035e-07, 5.5122467e-08,
        1.6145184e-04, 6.9911883e-05, 4.1101598e-06, 8.1579637e-04,
        1.8741899e-05, 1.1232085e-04]], dtype=float32)

In [34]:
model.predict([clean_text(tes1)]).argmax()

2

In [35]:
le.classes_[model.predict([clean_text(tes1)]).argmax()]

'daftar'

In [36]:
le.classes_[model.predict([clean_text(tes1)]).argmax()] == intent_json['intents'][2]['intent']

True

In [37]:
intent_json['intents'][2]['response']

['Untuk mengikuti rekrutmen ini, mohon ikuti langkah berikut ya, Kak:\n1. Baca tata cara melamar di https://rekrutmen.fiktif.id/tatacara\n2. Buat akun di https://rekrutmen.fiktif.id/registrasi menggunakan alamat email dan nomor telepon aktif milik pribadi\n3. Lengkapi formulir pendaftaran dan upload seluruh dokumen yang dibutuhkan dan tunggu pengumuman tahap berikutnya di https://rekrutmen.fiktif.id/news atau email masing-masing']

In [38]:
bot_response('aku mau daftar rekrutmen')

[{'CV': 3.6499318e-06}, {'biaya': 0.00020937042}, {'daftar': 0.957381}, {'dokumen': 0.0026261874}, {'error upload': 1.7256076e-06}, {'ktp': 0.00014831529}, {'link sosmed': 5.614712e-05}, {'lowongan': 0.00017104638}, {'lupa password': 2.531852e-05}, {'nama': 0.00092375744}, {'pas foto': 0.0001931675}, {'pengantar': 3.4116172e-06}, {'penutup': 3.6567333e-06}, {'qualification': 0.037070073}, {'responsibilities': 7.1733035e-07}, {'salary': 5.5122467e-08}, {'sapa': 0.00016145184}, {'sertifikat': 6.991188e-05}, {'skck': 4.11016e-06}, {'timeline': 0.0008157964}, {'training': 1.87419e-05}, {'transkrip-ijazah': 0.00011232085}]
daftar
Untuk mengikuti rekrutmen ini, mohon ikuti langkah berikut ya, Kak:
1. Baca tata cara melamar di https://rekrutmen.fiktif.id/tatacara
2. Buat akun di https://rekrutmen.fiktif.id/registrasi menggunakan alamat email dan nomor telepon aktif milik pribadi
3. Lengkapi formulir pendaftaran dan upload seluruh dokumen yang dibutuhkan dan tunggu pengumuman tahap berikutnya 

### Saving Model

In [39]:
import pickle
pickle.dump(le, open('saved_model/encoder.pkl', 'wb'))

In [40]:
pickle.dump({'config': textvect.get_config(),
             'weights': textvect.get_weights()}
            , open("saved_model/textvect.pkl", "wb"))

In [41]:
model.save('saved_model/model', save_traces=True)

INFO:tensorflow:Assets written to: saved_model/model\assets


INFO:tensorflow:Assets written to: saved_model/model\assets


## Inference

In [42]:
textvect('ok')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([267,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [43]:
clean_text('ok')

'ok'

In [44]:
textvect('ok').numpy().max() > 1

True

In [45]:
bot_response('a')

[{'CV': 0.047842514}, {'biaya': 0.04942192}, {'daftar': 0.08601724}, {'dokumen': 0.010020016}, {'error upload': 0.001243502}, {'ktp': 0.013547988}, {'link sosmed': 0.00034736114}, {'lowongan': 0.43180516}, {'lupa password': 0.026689481}, {'nama': 0.1665461}, {'pas foto': 0.0658423}, {'pengantar': 4.6225792e-05}, {'penutup': 0.00082994235}, {'qualification': 0.045604862}, {'responsibilities': 0.0017073184}, {'salary': 0.0005130706}, {'sapa': 0.011233536}, {'sertifikat': 0.014310752}, {'skck': 0.003714205}, {'timeline': 0.0043190112}, {'training': 0.0002831681}, {'transkrip-ijazah': 0.018114274}]
lowongan
Maaf, Kak. Aku tidak mengerti...


In [46]:
bot_response('ya')

[{'CV': 9.3230345e-05}, {'biaya': 0.00041688807}, {'daftar': 1.4174524e-05}, {'dokumen': 0.00028086244}, {'error upload': 5.276974e-08}, {'ktp': 4.4857857e-06}, {'link sosmed': 0.0012552438}, {'lowongan': 6.2103674e-05}, {'lupa password': 4.8935203e-07}, {'nama': 8.971853e-05}, {'pas foto': 1.1903045e-05}, {'pengantar': 0.00014197978}, {'penutup': 0.9900686}, {'qualification': 4.232843e-06}, {'responsibilities': 0.000111525944}, {'salary': 0.00047430245}, {'sapa': 0.00029212565}, {'sertifikat': 1.858246e-07}, {'skck': 4.917968e-06}, {'timeline': 0.0050283424}, {'training': 0.0016361757}, {'transkrip-ijazah': 8.354385e-06}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [47]:
testext = 'jeruk'
clean_text(testext), textvect(clean_text(testext))

('jeruk',
 <tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>)

In [48]:
textvect(clean_text('laper pengen makan jeruk')).numpy().max() > 1

False

In [49]:
bot_response('oh ok')

[{'CV': 0.00012652249}, {'biaya': 2.6025142e-05}, {'daftar': 6.9787177e-07}, {'dokumen': 0.0010799901}, {'error upload': 1.4099629e-06}, {'ktp': 2.302944e-05}, {'link sosmed': 0.00041731502}, {'lowongan': 1.24246835e-05}, {'lupa password': 4.5507644e-08}, {'nama': 2.6144748e-05}, {'pas foto': 5.451497e-07}, {'pengantar': 0.00017011147}, {'penutup': 0.99212945}, {'qualification': 2.7802223e-06}, {'responsibilities': 2.3791083e-05}, {'salary': 6.913265e-05}, {'sapa': 1.704399e-05}, {'sertifikat': 3.6298766e-07}, {'skck': 1.4476653e-05}, {'timeline': 0.00012657173}, {'training': 0.0057316255}, {'transkrip-ijazah': 5.481896e-07}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [50]:
bot_response('ada lowongan apa nih di pt fiktif')

[{'CV': 0.0002414947}, {'biaya': 0.00087421254}, {'daftar': 0.0047578844}, {'dokumen': 0.005162675}, {'error upload': 0.00021321345}, {'ktp': 4.269608e-05}, {'link sosmed': 0.0027397862}, {'lowongan': 0.89379126}, {'lupa password': 0.0036681888}, {'nama': 0.00074775616}, {'pas foto': 0.001782132}, {'pengantar': 2.4382167e-05}, {'penutup': 9.852817e-06}, {'qualification': 0.07416442}, {'responsibilities': 0.00210998}, {'salary': 7.6103825e-05}, {'sapa': 0.0008199398}, {'sertifikat': 0.0024317147}, {'skck': 0.0048006736}, {'timeline': 0.00026628308}, {'training': 7.3127594e-06}, {'transkrip-ijazah': 0.0012680179}]
lowongan
Saat ini PT Fiktif sedang membuka lowongan untuk posisi Data Analyst, Data Scientist, dan Data Engineer. Yuk daftar lewat https://rekrutmen.fiktif.id/


In [51]:
bot_response('ingin')

[{'CV': 0.0006396031}, {'biaya': 0.043710046}, {'daftar': 0.001498491}, {'dokumen': 0.00053036015}, {'error upload': 2.9588472e-07}, {'ktp': 1.5209243e-05}, {'link sosmed': 0.0027411818}, {'lowongan': 0.0012655057}, {'lupa password': 0.00039797404}, {'nama': 0.0008609855}, {'pas foto': 0.00085367647}, {'pengantar': 0.00063521997}, {'penutup': 0.030765012}, {'qualification': 0.00049248117}, {'responsibilities': 0.0021254572}, {'salary': 0.0016581819}, {'sapa': 0.7219143}, {'sertifikat': 4.490714e-06}, {'skck': 2.7255397e-05}, {'timeline': 0.07442133}, {'training': 0.11516344}, {'transkrip-ijazah': 0.00027952986}]
sapa
Maaf, Kak. Aku tidak mengerti...


In [52]:
bot_response('pas foto hilang gimana ya')

[{'CV': 0.010454456}, {'biaya': 0.0007756159}, {'daftar': 0.000109488385}, {'dokumen': 3.862607e-05}, {'error upload': 1.5368205e-06}, {'ktp': 5.2935782e-05}, {'link sosmed': 2.3943501e-05}, {'lowongan': 0.00092982757}, {'lupa password': 8.67447e-05}, {'nama': 0.0005225215}, {'pas foto': 0.97983813}, {'pengantar': 0.00040879767}, {'penutup': 4.7553214e-05}, {'qualification': 8.734105e-05}, {'responsibilities': 0.0052101724}, {'salary': 0.00013049148}, {'sapa': 0.00017100792}, {'sertifikat': 6.2571544e-06}, {'skck': 5.3874905e-06}, {'timeline': 0.0008712072}, {'training': 1.4528187e-05}, {'transkrip-ijazah': 0.00021350298}]
pas foto
Pas foto merupakan dokumen wajib pendaftaran ya, Kak.

Berikut ketentuan pas foto yang harus dipenuhi:
- Background berwarna biru dengan pakaian formal (kemeja dan/atau jas)
- Foto berukuran 3x4 dan berwarna
- Disarankan untuk menggunakan foto terbaru
- Upload foto di https://rekrutmen.fiktif.id/dokumen dengan ukuran file tidak lebih dari 1 MB dengan format 

ktp, pas foto, password masih ketuker2

In [53]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [54]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([133,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [55]:
textvect('wis')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>

In [56]:
data = {'user_input':'hai hai'}
data = json.dumps(data)
loaded_data = json.loads(data)

In [57]:
type(loaded_data)

dict

In [58]:
tespred = str(model.predict([clean_text(loaded_data['user_input'])]).argmax())

In [59]:
tes_res = {'pred':tespred}
tes_res = json.dumps(tes_res)
loaded_res = json.loads(tes_res)

In [60]:
le.classes_[int(loaded_res['pred'])]

'sapa'

In [61]:
loaded_data['user_input']

'hai hai'

In [62]:
1000

1000

In [63]:
str(1000)

'1000'

In [64]:
int(str(1000))

1000

In [65]:
le.classes_[int(str(10))]

'pas foto'

In [66]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([133,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [67]:
user_input = 'apa kualifikasi engineer'
clean_input = clean_text(user_input)
pred = model.predict([clean_input]).argmax()
if textvect(clean_input).numpy().max() <= 1:
    label_idx = 1000
else:
    label_idx = pred 
if label_idx != 1000:
    i = 0
    while i < len(intent_json['intents']):
        if le.classes_[label_idx] == intent_json['intents'][i]['intent'] and (le.classes_[label_idx] != 'responsibilities' or le.classes_[label_idx] != 'qualification'):
            response = intent_json['intents'][i]['response']
            break
        elif le.classes_[label_idx] == 'responsibilities':
            if any(x in clean_input for x in ['scientist', 'ds']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Scientist:\n- Merancang dan mengembangkan berbagai solusi Machine Learning dan Deep Learning untuk meningkatkan pengalaman pengguna bagi konsumen\n- Berkolaborasi dengan seluruh elemen bisnis dan bertanggung jawab untuk merencanakan solusi end-to-end berbasis data untuk menyelesaikan permasalahan bisnis\n- Menjadi thinking partner bagi stakeholder lain untuk memperbaiki alur perjalanan data dan penggunaannya dalam operasional perusahaan, misal merancang proses feedback-loop atau human-in-the-loop untuk meningkatkan performa model secara berkelanjutan"]
                break
            elif any(x in clean_input for x in ['engineer', 'de']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Engineer:\n- Membangun dan menjaga end-to-end data pipeline dari input dan output heterogen\n- Menangani dan mengelola data warehouse\n- Membantu tim mentransformasikan data (ETL) dan mengembangkan proses ETL dari beberapa sumber\n- Menganalisis dan mengorganisir data mentah \n- Memastikan kualitas data dan integrasi data"]
                break
            elif any(x in clean_input for x in ['analyst', 'da', 'analis']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Analyst:\n- Mengumpulkan dan menyediakan data untuk membantu stakeholder lain meningkatkan metrik bisnis perusahaan dan retensi pelanggan\n- Menganalisis data untuk menemukan insight yang dapat ditindaklanjuti seperti membuat funnel conversion analysis, cohort analysis, long-term trends, user segmentation, dan dapat membantu meningkatkan kinerja perusahaan dan mendukung pengambilan keputusan yang lebih baik\n - Mengidentifikasi kebutuhan dan peluang bisnis berdasarkan data yang tersedia"]
                break
        elif le.classes_[label_idx] == 'qualification':
            if any(x in clean_input for x in ['scientist', 'ds']):
                response = ["Untuk posisi Data Scientist ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik, Machine Learning, Deep Learning untuk menyelesaikan permasalahan bisnis\n3. Memiliki pengalaman kerja di bidang Data Science selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif any(x in clean_input for x in ['engineer', 'de']):
                response = ["Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.\n3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun\n4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif any(x in clean_input for x in ['analyst', 'da', 'analis']):
                response = ["Untuk posisi Data Analyst ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik dan Inferensial untuk mencari peluang bisnis\n3. Memiliki pengalaman kerja di bidang Data Analyst selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data serta visualisasi dengan tools seperti Power BI, Tableau, dll.\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
        else:
            i+=1
else:
    response = ['Maaf, Kak. Aku tidak mengerti chatnya...']

print(np.random.choice(response))

Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:
1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan
2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.
3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun
4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL
5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik


In [68]:
le.classes_[label_idx]

'qualification'